In [1]:
import os, openai

In [2]:
with open('./sample_data/token.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
client = openai.OpenAI()

In [5]:
content = """다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.
문장: \"나는 빨간 사과가 좋더라\n
"""

In [6]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
)

In [7]:
print(response.choices[0].message.content)

```json
{
  "color": "red"
}
```


In [8]:
content = """다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.
문장: \"아침에는 인간적으로 커피 한 잔 주고 뭐라 하자\n
"""

In [9]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
)

In [10]:
print(response.choices[0].message.content)

해당 문장에는 색깔에 대한 정보가 포함되어 있지 않습니다. 

따라서 색깔 정보를 추출할 수 없기 때문에 빈 JSON 객체를 반환하겠습니다.

```json
{}
```


In [11]:
schema = {
  "$schema": "http://json-schema.org/draft-07/schema",
  "type": "object",
  "properties": {
    "color": {
      "type": "string"
    }
  },
  "required": ["color"]
}

In [12]:
content = f"""다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.
               문장: '나는 빨간 사과가 좋더라'
               json schema 는 다음과 같다 : {schema}
               색깔 정보가 없으면 no color 로 대신해라
            """

In [13]:
# 이전 방식
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": f"{content}"
    }
  ],
)

In [14]:
print(response.choices[0].message.content)

```json
{
  "$schema": "http://json-schema.org/draft-07/schema",
  "type": "object",
  "properties": {
    "color": {
      "type": "string"
    }
  },
  "required": [
    "color"
  ]
}
```

```json
{
  "color": "red"
}
```

문장: '나는 빨간 사과가 좋더라'에서 '빨간'이라는 색깔 정보를 추출했습니다. 영어로 'red'라고 번역하여 JSON 포맷으로 만들었습니다.


잘못된 출력의 예:

- 틀린 아웃풋 (output 텍스트가 포함됨): output: {‘color’: ‘blue’}
- 틀린 아웃풋 : 사용자는 노란색을 좋아합니다.
- 틀린 아웃풋: 주신 내용에서는 색깔 정보를 찾을 수가 없는데 다른 더 도와드릴 일이 있나요?

In [15]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[{"role": "user", "content": "나는 노란색이 좋더라"}],
  functions=[
      {
          "name": "color_extractor",
          "description": "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력함",
          "parameters": {
              "type": "object",
              "properties": {
                  "color": {
                      "type": "string",
                      "description": "색깔 정보를 빼내와서 그에 해당하는 영어 단어를 출력함",
                  },
              },
              "required": ["color"],
          },
      }
  ]
)

In [16]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"color":"노란색"}', name='color_extractor'), tool_calls=None)


In [17]:
print(response.choices[0].message.function_call.arguments)

{"color":"노란색"}


In [18]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[{"role": "user", "content": "퇴근했는데 업무 톡 날리는 양심없는 인간"}],
  functions=[
      {
          "name": "color_extractor",
          "description": "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력함",
          "parameters": {
              "type": "object",
              "properties": {
                  "color": {
                      "type": "string",
                      "description": "색깔 정보를 빼내와서 그에 해당하는 영어 단어를 출력함",
                  },
              },
              "required": ["color"],
          },
      }
  ]
)

In [19]:
print(response.choices[0].message)

ChatCompletionMessage(content='업무 관련된 메시지를 퇴근 후에 받는 것은 정말 스트레스 받을 수 있죠. 퇴근 후의 시간은 휴식과 개인적인 일에 집중할 수 있는 소중한 시간입니다. 그런 경우 어떻게 대처하는 것이 좋을지 생각해보신 적 있나요?', role='assistant', function_call=None, tool_calls=None)


In [20]:
print(response.choices[0].message.content)

업무 관련된 메시지를 퇴근 후에 받는 것은 정말 스트레스 받을 수 있죠. 퇴근 후의 시간은 휴식과 개인적인 일에 집중할 수 있는 소중한 시간입니다. 그런 경우 어떻게 대처하는 것이 좋을지 생각해보신 적 있나요?


In [21]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "아침에 세 개 저녁에 네개"}],
    functions=[
        {
            "name": "apple_add",
            "description": "사과 숫자를 더함",
            "parameters": {
                "type": "object",
                "properties": {
                    "quantity1": {
                        "type": "integer",
                        "description": "첫번째 숫자",
                    },
                    "quantity2": {
                        "type": "integer",
                        "description": "두번째 숫자",
                    },
                },
                "required": ["quantity1", "quantity2"],
            },
        }
    ],
    function_call="auto",
)

In [22]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"quantity1":3,"quantity2":4}', name='apple_add'), tool_calls=None)


In [23]:
print(response.choices[0].message.function_call.arguments)

{"quantity1":3,"quantity2":4}


In [24]:
import openai
import json
from duckpy import Client
import ast

In [25]:
duckduckgo_client = Client()

In [26]:
def duck_search(query) -> str:
    """Runs a duckduckgo search"""
    output = duckduckgo_client.search(query)
    return str(output)

In [27]:
completion = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        functions=[
            {
                "name": "duck_search",
                "description": "Used to search online",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Translate the Korean content into English input query",
                        },
                    },
                    "required": ["query"],
                },
            }
        ],
        messages=[
            {"role": "system", "content": "You must use the `duck_search` function to get information."},
            {"role": "user", "content": "조지 클루니 생일이 언제야?"},
        ]
    )

In [28]:
print(completion)

ChatCompletion(id='chatcmpl-9m0OD2AdOCdxuc9P9NPUfVsFILKbx', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"query":"George Clooney birthday"}', name='duck_search'), tool_calls=None))], created=1721228269, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_5e997b69d8', usage=CompletionUsage(completion_tokens=18, prompt_tokens=76, total_tokens=94))


In [29]:
message = completion.choices[0].message

In [30]:
if message.function_call:
    function_name = message.function_call.name
    args = ast.literal_eval(message.function_call.arguments)
    function_response = duck_search(
        query=args.get("query"),
    )
    print(function_response)
    completion_final = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": "조지 클루니 생일이 언제야?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )
    print(completion_final.choices[0].message.content)

[{'title': 'George Clooney - Wikipedia', 'description': 'George Timothy Clooney (born May 6, 1961) is an American actor and filmmaker. Known for his work in both blockbuster and independent films, he has received numerous accolades, including two Academy Awards, a British Academy Film Award and four Golden Globe Awards.His honors include the Cecil B. DeMille Award, the Honorary César, the AFI Life Achievement Award, and the Kennedy Center Honor.', 'url': 'm/l/?uddg=https://en.wikipedia.org/wiki/George_Clooney&rut=37382dd57f2f94af2bc9d8e9772cbfa1c96bb82b8d192c2706f7a84706bc9542'}, {'title': 'George Clooney - Movies, Wife & Age - Biography', 'description': 'George Timothy Clooney was born on May 6, 1961, in Lexington, Kentucky, into a well-known family of media and entertainment personalities. His father, Nick, spent many years as a television ...', 'url': 'm/l/?uddg=https://www.biography.com/actors/george-clooney&rut=2fcc7473ace13a78cb9eb599ebd7a05b1d5d825214f28dcc175db37c3db28d63'}, {'

In [31]:
example_user_input = "런던 날씨가 안 좋다면서?"

In [32]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": example_user_input}],
    functions=[
      {
          "name": "get_different_responses",
          "description": "답변을 여러가지 말투로 하기",
          "parameters": {
              "type": "object",
              "properties": {
                  "response_up": {
                      "type": "string",
                      "description": "답변을 높임말로 바꿈",
                  },
                  "response_down": {
                      "type": "string",
                      "description": "답변을 반말로 바꿈",
                  }
              },
              "required": ["response_up", "response_down"],
          },
      }
    ],
    function_call={"name": "get_different_responses"},
)


In [33]:
reply_content = completion.choices[0].message
print(reply_content.function_call.arguments)

{"response_up":"네, 요즘 런던은 우중충한 날씨가 많아요.","response_down":"어, 요즘 런던 날씨가 별로야."}


In [34]:
example_user_input = \
"""레드햇 리눅스에서 새로운 사용자 만들고
그 사용자에게 /usr/arb 폴더 읽기 쓰기 사용권한을 주려면 어떻게 해야 해?"""

In [35]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": example_user_input}],
    functions=[
      {
        "name": "get_commands",
        "description": "Get a list of bash commands on an RedHat machine",
        "parameters": {
            "type": "object",
            "properties": {
                "commands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "description": "A terminal command string"
                    },
                    "description": "List of terminal command strings to execute"
                }
            },
            "required": ["commands"]
        }
      },
    ],
    function_call={"name": "get_commands"},
)

In [36]:
print(completion.choices[0].message.function_call.arguments)

{"commands":["sudo useradd newuser","sudo usermod -aG wheel newuser","sudo mkdir -p /usr/arb","sudo chown newuser:newuser /usr/arb","sudo chmod 755 /usr/arb"]}


In [37]:
import ast

In [38]:
example_user_input = "주말에 친구들 오는데 뭐 먹지? 소고기랑 감자랑 밀가루 있어"

In [39]:
def get_recipes_with_ingredients(ingredients):
  return f"{ingredients} 를 다 볶아서 볶음밥으로 만들어 먹읍시다!"

In [40]:
completion =  client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": example_user_input}],
    functions=[
      {
          "name": "get_recipes_with_ingredients",
          "description": "주어진 텍스트에서 음식을 만들 수 있는 재료 이름을 뽑아낸다",
          "parameters": {
              "type": "object",
              "properties": {
                  "ingredients": {
                      "type": "string",
                      "description": "음식 재료",
                  },
              },
              "required": ["ingredients"],
          },
      }
    ],
    function_call={"name": "get_recipes_with_ingredients"},
)

In [41]:
message = completion.choices[0].message

In [42]:
if message.function_call:
    function_name = message.function_call.name
    args = ast.literal_eval(message.function_call.arguments)
    function_response = (
          get_recipes_with_ingredients(ingredients=args.get("ingredients"))),
    completion_final = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": example_user_input},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": str(function_response),
            },
        ],
    )
    print(completion_final.choices[0].message.content)

주말에 친구들을 초대해서 맛있는 소고기와 감자를 활용한 볶음밥을 만들어 보는 건 어떨까요? 아래에 간단한 레시피를 소개할게요.

### 소고기 감자 볶음밥 레시피

#### 재료
- 소고기: 200g
- 감자: 2개
- 밀가루: 2큰술
- 밥: 2공기
- 양파: 1/2개 (선택사항)
- 간장: 2큰술
- 설탕: 1큰술
- 다진 마늘: 1작은술
- 후추: 약간
- 소금: 약간
- 식용유: 2큰술
- 참기름: 1큰술 (선택사항)
- 대파: 약간 (선택사항)

#### 준비
1. 소고기는 얇게 썰고, 간장 1큰술, 설탕 1/2큰술, 다진 마늘, 후추로 재워줍니다.
2. 감자는 껍질을 벗기고 작은 크기로 깍둑썰기 합니다.
3. 양파와 대파는 잘게 썰어둡니다.

#### 요리 과정
1. 팬에 식용유를 두르고, 재워둔 소고기를 볶아줍니다.
2. 소고기가 반 정도 익으면 감자와 양파를 넣고 볶아줍니다.
3. 감자가 어느 정도 익으면 밥을 넣고 잘 섞어줍니다.
4. 간장 1큰술과 설탕 1/2큰술, 소금, 후추로 간을 맞춥니다.
5. 마지막으로 참기름을 넣고 다시 한 번 섞어줍니다.
6. 다 볶아지면 대파를 뿌려 데코레이션합니다.

이렇게 소고기와 감자를 메인 재료로 만든 볶음밥은 간단하면서도 맛있게 즐길 수 있는 메뉴입니다. 즐거운 시간 보내세요! 😊


In [43]:
print(completion.choices[0].message.function_call.arguments)

{"ingredients":"소고기, 감자, 밀가루"}


In [44]:
import textwrap as tr

In [47]:
my_str = completion_final.choices[0].message.content
lines = tr.wrap(my_str, width=60)
[print(x) for x in lines]

print(completion_final.choices[0].message.content)

주말에 친구들을 초대해서 맛있는 소고기와 감자를 활용한 볶음밥을 만들어 보는 건 어떨까요? 아래에 간단한
레시피를 소개할게요.  ### 소고기 감자 볶음밥 레시피  #### 재료 - 소고기: 200g - 감자:
2개 - 밀가루: 2큰술 - 밥: 2공기 - 양파: 1/2개 (선택사항) - 간장: 2큰술 - 설탕: 1큰술
- 다진 마늘: 1작은술 - 후추: 약간 - 소금: 약간 - 식용유: 2큰술 - 참기름: 1큰술 (선택사항)
- 대파: 약간 (선택사항)  #### 준비 1. 소고기는 얇게 썰고, 간장 1큰술, 설탕 1/2큰술, 다진
마늘, 후추로 재워줍니다. 2. 감자는 껍질을 벗기고 작은 크기로 깍둑썰기 합니다. 3. 양파와 대파는 잘게
썰어둡니다.  #### 요리 과정 1. 팬에 식용유를 두르고, 재워둔 소고기를 볶아줍니다. 2. 소고기가 반
정도 익으면 감자와 양파를 넣고 볶아줍니다. 3. 감자가 어느 정도 익으면 밥을 넣고 잘 섞어줍니다. 4.
간장 1큰술과 설탕 1/2큰술, 소금, 후추로 간을 맞춥니다. 5. 마지막으로 참기름을 넣고 다시 한 번
섞어줍니다. 6. 다 볶아지면 대파를 뿌려 데코레이션합니다.  이렇게 소고기와 감자를 메인 재료로 만든
볶음밥은 간단하면서도 맛있게 즐길 수 있는 메뉴입니다. 즐거운 시간 보내세요! 😊
주말에 친구들을 초대해서 맛있는 소고기와 감자를 활용한 볶음밥을 만들어 보는 건 어떨까요? 아래에 간단한 레시피를 소개할게요.

### 소고기 감자 볶음밥 레시피

#### 재료
- 소고기: 200g
- 감자: 2개
- 밀가루: 2큰술
- 밥: 2공기
- 양파: 1/2개 (선택사항)
- 간장: 2큰술
- 설탕: 1큰술
- 다진 마늘: 1작은술
- 후추: 약간
- 소금: 약간
- 식용유: 2큰술
- 참기름: 1큰술 (선택사항)
- 대파: 약간 (선택사항)

#### 준비
1. 소고기는 얇게 썰고, 간장 1큰술, 설탕 1/2큰술, 다진 마늘, 후추로 재워줍니다.
2. 감자는 껍질을 벗기고 작은 크기로 깍둑썰기 합니다.
3. 양파와